In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf

/tmp/ipykernel_6027/2864461462.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-05-23 08:35:19.109321: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 08:35:19.264957: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 08:35:

In [2]:
data = pd.read_table('../data/u.data')

In [3]:
data

,user,event,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
users = data['user'].unique()
events = data['event'].unique()
user_to_idx = {user: idx for idx, user in enumerate(users)}
event_to_idx = {event: idx for idx, event in enumerate(events)}

data['user_idx'] = data['user'].map(user_to_idx)
data['event_idx'] = data['event'].map(event_to_idx)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=21)

In [5]:
class NCF(tf.keras.Model):
    def __init__(self, num_users, num_events, embedding_dim=96):
        super(NCF, self).__init__()
        self.user_embed = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.event_embed = tf.keras.layers.Embedding(num_events, embedding_dim)
        self.mlp = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='swish'),
            tf.keras.layers.Dense(64, activation='swish'),
            # tf.keras.layers.Dense(64, activation='swish'),
            tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):
        user_idx, event_idx = inputs
        user_emb = self.user_embed(user_idx)
        event_emb = self.event_embed(event_idx)
        concat_emb = tf.concat([user_emb, event_emb], axis=1)
        output = self.mlp(concat_emb)
        return output


num_users = len(users)
num_events = len(events)
model = NCF(num_users, num_events)

model.compile(optimizer='nadam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

2024-05-21 12:54:14.600307: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-21 12:54:14.658606: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-21 12:54:14.658759: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
train_user_idx = tf.constant(train_data['user_idx'].values, dtype=tf.int32)
train_event_idx = tf.constant(train_data['event_idx'].values, dtype=tf.int32)
train_target = tf.constant(train_data['rating'].values, dtype=tf.float32)

In [7]:
train_user_idx

<tf.Tensor: shape=(80000,), dtype=int32, numpy=array([ 70, 925, 784, ..., 837, 739,  53], dtype=int32)>

In [8]:
train_event_idx

<tf.Tensor: shape=(80000,), dtype=int32, numpy=array([372, 262, 355, ..., 454, 229, 269], dtype=int32)>

In [9]:
model.fit([train_user_idx, train_event_idx], train_target, validation_split=0.1, epochs=100, batch_size=128, callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)]
)

Epoch 1/100


2024-05-21 12:54:15.640526: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-21 12:54:15.937623: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe0ee4c8700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-21 12:54:15.937645: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-05-21 12:54:15.952644: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-21 12:54:15.966699: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1716276256.081724    5946 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


563/563 [==============================] - 8s 12ms/step - loss: 1.5851 - root_mean_squared_error: 1.2590 - val_loss: 0.9253 - val_root_mean_squared_error: 0.9619
Epoch 2/100
563/563 [==============================] - 2s 3ms/step - loss: 0.9031 - root_mean_squared_error: 0.9503 - val_loss: 0.9075 - val_root_mean_squared_error: 0.9527
Epoch 3/100
563/563 [==============================] - 2s 3ms/step - loss: 0.8815 - root_mean_squared_error: 0.9389 - val_loss: 0.8911 - val_root_mean_squared_error: 0.9440
Epoch 4/100
563/563 [==============================] - 2s 3ms/step - loss: 0.8592 - root_mean_squared_error: 0.9270 - val_loss: 0.8820 - val_root_mean_squared_error: 0.9391
Epoch 5/100
563/563 [==============================] - 2s 3ms/step - loss: 0.8434 - root_mean_squared_error: 0.9183 - val_loss: 0.8836 - val_root_mean_squared_error: 0.9400
Epoch 6/100
563/563 [==============================] - 1s 3ms/step - loss: 0.8283 - root_mean_squared_error: 0.9101 - val_loss: 0.8750 - val_root_

In [10]:

test_user_idx = tf.constant(test_data['user_idx'].values, dtype=tf.int32)
test_event_idx = tf.constant(test_data['event_idx'].values, dtype=tf.int32)
test_target = tf.constant(test_data['rating'].values, dtype=tf.float32)

test_loss = model.evaluate([test_user_idx, test_event_idx], test_target)

  1/625 [..............................] - ETA: 9s - loss: 0.9777 - root_mean_squared_error: 0.9888

625/625 [==============================] - 1s 1ms/step - loss: 0.8456 - root_mean_squared_error: 0.9196


In [ ]:
import tensorflow as tf

# Assuming event_data contains all unique event IDs
# Example: event_data = [0, 1, 2, ..., num_events-1]
num_events = len(events)  # Replace with the actual number of events
user_id = 22

# Create tensors for the user and all possible events
user_ids = tf.constant([user_id] * num_events, dtype=tf.int32)
event_ids = tf.constant(range(num_events), dtype=tf.int32)

# Predict ratings for all events for the user
predictions = model.predict([user_ids, event_ids])

# Convert predictions to a tensor and ensure it's a 1D tensor
predictions = tf.constant(predictions, dtype=tf.float32)
predictions = tf.reshape(predictions, [-1])

# Get the top K recommendations, for example, top 10
K = 10
if len(predictions) < K:
    raise ValueError(f"Number of events ({len(predictions)}) is less than K ({K})")

top_k_values, top_k_indices = tf.math.top_k(predictions, k=K)

# Retrieve the event IDs for the top K predictions
top_k_event_ids = tf.gather(event_ids, top_k_indices).numpy()

print(f"Top {K} recommendations for user {user_id}: {top_k_event_ids}")


In [7]:
user_ids = tf.constant([user_id] * num_events, dtype=tf.int32)
event_ids = tf.constant(range(num_events), dtype=tf.int32)
print(event_ids)

tf.Tensor([22 22 22 ... 22 22 22], shape=(1682,), dtype=int32)
